In [1]:
import psycopg2
import pandas as pd

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# SQL query to create the new table
create_table_query = """
CREATE TABLE IF NOT EXISTS aqi_summary (
    id SERIAL PRIMARY KEY,
    uid INTEGER,
    current_aqi INTEGER,
    avg_aqi_last_hour FLOAT,
    avg_aqi_last_day FLOAT,
    latitude FLOAT,
    longitude FLOAT,
    current_aqi_timestamp TIMESTAMP
);
"""

# Execute the create table query
cursor.execute(create_table_query)
conn.commit()

In [2]:
# SQL query to select data for the new table
select_data_query = """
SELECT
    aqi_data.uid,
    aqi_data.aqi AS current_aqi,
    (SELECT AVG(sub.aqi)
        FROM aqi_data sub
        WHERE sub.uid = aqi_data.uid
        AND sub.timestamp >= aqi_data.timestamp - INTERVAL '1 hour'
        AND sub.timestamp <= aqi_data.timestamp
    ) AS avg_aqi_last_hour,
    (SELECT AVG(sub.aqi)
        FROM aqi_data sub
        WHERE sub.uid = aqi_data.uid
        AND sub.timestamp >= aqi_data.timestamp - INTERVAL '1 day'
        AND sub.timestamp <= aqi_data.timestamp
    ) AS avg_aqi_last_day,
    station_info.latitude,
    station_info.longitude,
    aqi_data.timestamp AS current_aqi_timestamp
FROM
    aqi_data
JOIN
    station_info ON aqi_data.uid = station_info.uid
WHERE
    aqi_data.timestamp = (
        SELECT MAX(timestamp)
        FROM aqi_data AS sub
        WHERE sub.uid = aqi_data.uid
    );
"""

# SQL query to insert data into the new table
insert_data_query = """
"""

# Execute the select data query
cursor.execute(select_data_query)
result = cursor.fetchall()

# Convert the result to a pandas DataFrame for easier inspection
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=columns)

# Display the DataFrame
print(df)

# # Execute the insert data query
# cursor.execute(insert_data_query)
# conn.commit()

      uid  current_aqi    avg_aqi_last_hour     avg_aqi_last_day   latitude  \
0   10482         34.0  34.0000000000000000  34.0000000000000000   6.624747   
1    8322         88.0  88.0000000000000000  88.0000000000000000  13.054551   
2   13629         28.0  28.0000000000000000  28.0000000000000000   8.426923   
3    1826         34.0  34.0000000000000000  34.0000000000000000   9.126017   
4   13439         44.0  44.0000000000000000  44.0000000000000000  21.002383   
..    ...          ...                  ...                  ...        ...   
74   1823         88.0  88.0000000000000000  88.0000000000000000  18.278333   
75   9556         48.0  48.5000000000000000  48.5000000000000000   6.147431   
76   9587         73.0  70.5000000000000000  70.5000000000000000  19.858547   
77  13576         76.0  77.0000000000000000  77.0000000000000000  11.800270   
78   9450         48.0  63.0000000000000000  63.0000000000000000  19.687157   

     longitude current_aqi_timestamp  
0   100.0661

In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()